In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import DIM

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_positions = 10 #number of robot sample positions
n_complexity = 3 #model complexity

trajectory = Trajectory(n_complexity, dim=DIM, model='polynomial')
environment = Environment(n_anchors)

basis = trajectory.get_basis(n_samples=n_positions)

environment.set_random_anchors(seed=2)
plt.figure()
environment.plot()
trajectory.plot(basis)

A few first observations from cell below: 

* looks like when we use exactly K*D measurements, such as with strategy 'simple', there is almost never a unique solution.
* when we add one more measurements, we get more unique solutions. 
* 

In [ ]:
n_it = 12

## Problem: this actually returns garbage if the first estimate is not feasible. 
## See here: https://github.com/scipy/scipy/issues/7618
#method = 'minimize'  

# Problem: this returns the least squares estimate, but does not impose the quadratic constraints. 
# the residuals of this might be non-zero.
method = 'least_squares'

# Problem: produces memory error even for small problem size. 
#method = 'grid'

# Problem: can only take into account exactly K*dim constraints. 
#method = 'roots'

# the first point sees dim+1 anchors, the next K-2 points see the first two anchors, the Kth point sees only one. 
strategy = 'simple'


for i in range(n_it):
    # create random trajectory
    trajectory.set_coeffs(seed=i)
    basis = trajectory.get_basis(n_samples=n_positions)
    sample_points = trajectory.get_sampling_points(basis=basis)

    D_complete = get_D_topright(environment.anchors, sample_points)
    mask = create_mask(n_positions, n_anchors, strategy=strategy, dim=DIM, n_complexity=n_complexity, seed=i)
    D_missing = np.multiply(D_complete, mask)

    try:
        X = exactSolution(D_missing, environment.anchors, basis, method, verbose=True)
        assert np.allclose(X, trajectory.coeffs, atol=1e-3)
        
        #X_noiseless = exactSolution(D_complete, environment.anchors, basis, method)
        #assert np.allclose(X_noiseless, trajectory.coeffs)
        print('Seed {} ok.'.format(i))
        
    except ValueError as e:
        print('ValueError for seed {}:{}'.format(i, e))
        
    except AssertionError:
        print('Result not exact for seed:', i)
        #print('diff noiseless ok:', np.sum(np.abs(X_noiseless - trajectory.coeffs)) < 1e-10)
        print('diff noisy:    ', X - trajectory.coeffs)
        break
        
    except Exception as e:
        print('Singular matrix for seed {}? Error message:'.format(i)) 
        raise e

from exact_solution import objective_root
print('objective_root found:')
print(objective_root(X, environment.anchors, basis, D_missing))
print('objective_root original:')
print(objective_root(trajectory.coeffs, environment.anchors, basis, D_missing))

In [ ]:
traj_second_solution = trajectory.copy()
traj_second_solution.set_coeffs(coeffs=X)

plt.figure()
plt.title('plot of two ambiguous solutions')

trajectory.plot(basis, mask=mask, color='green')
trajectory.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
trajectory.plot_number_measurements(basis, mask, legend=True)

traj_second_solution.plot(basis, mask=mask, color='red')
traj_second_solution.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
traj_second_solution.plot_number_measurements(basis, mask, legend=False)
environment.plot()
environment.annotate()

plt.axis('equal')
#plt.xlim([0, 6])
#plt.ylim([0, 6])